In [3]:
import sys
import os
import numpy as np
import pandas as pd
sys.path.append('/Users/yuxiangyu/Documents/毕业论文-20171126/毕业论文code/')
from DataAnalysis import DataAnalysis
from sklearn import neighbors as knn
# 基本处理
stat = DataAnalysis()
now_time = stat.get_time()

In [4]:
# 获取数据集路径
train_data_path_list = stat.datasets

In [5]:
save_columns_name_list = ['k_value', 'KNN', 'delKNN', 'del_num', 'pre_del_num','del_attr', 'iter_cdf_mean', 'iter_std', 'data']

In [6]:
per_num = 0

In [38]:
train_data_path_list

['/Users/yuxiangyu/Documents/论文材料/实验室/Iris/iris.csv',
 '/Users/yuxiangyu/Documents/论文材料/实验室/Winequality/new-winequality-white.txt',
 '/Users/yuxiangyu/Documents/论文材料/实验室/Winequality/new-winequality-red.txt',
 '/Users/yuxiangyu/Documents/论文材料/实验室/Glass/new1-new-glass.txt',
 '/Users/yuxiangyu/Documents/论文材料/实验室/seeds/seeds_dataset.txt',
 '/Users/yuxiangyu/Documents/论文材料/实验室/breast_cancer/new-breast-cancer-wisconsin.txt',
 '/Users/yuxiangyu/Documents/论文材料/实验室/Wine/new-win.txt',
 '/Users/yuxiangyu/Documents/论文材料/实验室/balance_scale/new-balance-scale.txt',
 '/Users/yuxiangyu/Documents/论文材料/实验室/segmentation/new-segmentation.txt']

## 基本描述统计

In [58]:
tmp_data_df = stat.open_csv('/Users/yuxiangyu/Documents/论文材料/实验室/Winequality/new-winequality-red.txt')

In [59]:
set(tmp_data_df.label)

{3, 4, 5, 6, 7, 8}

In [60]:
tmp_data_df.shape

(1599, 12)

In [61]:
tmp_data_df.shape[0] * 0.8, tmp_data_df.shape[0] - tmp_data_df.shape[0] * 0.8

(1279.2, 319.79999999999995)

In [7]:
for ix, tmp_data_path in enumerate(train_data_path_list[per_num : per_num + 1]):
    last_result_df = pd.DataFrame(columns=save_columns_name_list)
    file_name = tmp_data_path.split('/')[-1].split('.')[0]
    print(file_name)
    # 读取数据
    data_df = stat.open_csv(tmp_data_path)

    # 行列数量
    row_len, col_len = data_df.shape
    # 生成相等权重的数组
    orginal_weight_array = np.array(list([1]* (col_len - 1)))
    orginal_weight_array = orginal_weight_array / sum(orginal_weight_array) 
    # 获取训练集和测试集
    train_data, test_data = stat.get_train_test(data_df)

    # 传统 KNN 算法
    train_array = train_data.drop('label', axis=1)
    test_array = test_data.drop('label', axis=1)
    for k_value in range(1, 21):
        clf = knn.KNeighborsClassifier(n_neighbors = k_value, weights = 'uniform')
        clf.fit(train_data.drop('label', axis = 1), train_data['label'])
        # 预测结果
        predict_label_array = clf.predict(test_array)
        classic_knn_result = stat.get_accuracy_score(predict_label_array, test_data['label'])
#         print('k值:{kvalue}, 分类准确率:{re:.2f} %'.format(kvalue = k_value,re = classic_knn_result * 100)) 
        last_result_df.loc['{data}:{k}'.format(data = file_name, k = k_value), 'k_value'] = k_value
        last_result_df.loc['{data}:{k}'.format(data = file_name, k = k_value), 'KNN'] = round(classic_knn_result * 100,2)
        last_result_df.loc['{data}:{k}'.format(data = file_name, k = k_value), 'data'] = file_name
 
    center_result_df = pd.DataFrame(columns=train_data.columns)
    iter_center_result_df = pd.DataFrame(columns=train_data.columns)

iris


/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [8]:
for class_name, values in train_data.groupby('label'):
    tmp_class_name_list = values.columns
    columns_name_list = [i for i in tmp_class_name_list if 'label' not in i]
    tmp_center_mean_list = list(values[columns_name_list].mean())
    tmp_center_mean_list.append(class_name)
    center_result_df.loc[class_name, :] = tmp_center_mean_list
    tmp_result,iter_n = stat.get_weights_iteration(values[columns_name_list], orginal_weight_array, error_param = 1e-4, max_iter= 10000, alpha=0.02)
    tmp_result.append(class_name)
    iter_center_result_df.loc['{name}:{iter}'.format(name = class_name, iter = iter_n), :] = tmp_result
iter_without_label = iter_center_result_df[list(iter_center_result_df.columns)[:-1]]
tmp_iter_std = iter_without_label.std()
tmp_iter_std.sort_values(inplace= True)

In [10]:
# iter_without_label.to_latex('Iris_实例.tex')

In [12]:
iter_without_label.std()

col_1    0.182955
col_2    0.275724
col_3    0.182955
col_4    0.317692
dtype: float64

In [14]:
iter_mean_value = iter_without_label.mean()

In [15]:
iter_mean_value

col_1    0.119933
col_2    0.171896
col_3    0.119933
col_4    0.588239
dtype: float64

In [16]:
iter_mean_value.sort_values(ascending = False, inplace= True)

In [21]:
tmp_df = iter_mean_value.to_frame(name='mean')

In [22]:
tmp_df

,mean
col_4,0.588239
col_2,0.171896
col_3,0.119933
col_1,0.119933


In [17]:
# 累计函数
cdf_cumsum_mean = iter_mean_value.cumsum()

In [24]:
tmp_df_one = cdf_cumsum_mean.to_frame('累计')

In [27]:
tmp_std_df = iter_without_label.std()

In [29]:
tmp_std_df = tmp_std_df.to_frame('标准差')

In [25]:
tmp_df_one

,累计
col_4,0.588239
col_2,0.760135
col_3,0.880067
col_1,1.000000


In [32]:
tmp_result = pd.concat([tmp_df, tmp_df_one,tmp_std_df], axis=1)

In [35]:
tmp_result.sort_values('累计', inplace=True)

In [37]:
tmp_result.to_latex('Iris_筛选.tex')

In [15]:
cdf_cumsum_mean_dict = cdf_cumsum_mean.to_dict()

In [16]:
# cdf_cumsum_mean_dict

In [17]:
tmp_args = 0.99999

In [18]:
# 贡献较低的数据
iter_left_series = iter_mean_value.loc[iter_mean_value.cumsum() >= tmp_args]

In [19]:
# 欲删除个数
prepare_del_num = len(iter_left_series)

In [20]:
prepare_del_num

5

In [21]:
tmp_iter_std_dict = tmp_iter_std.to_dict()

In [22]:
tmp_iter_std

col_2     4.571008e-08
col_13    1.794036e-06
col_9     2.285504e-06
col_15    2.285504e-06
col_17    2.285504e-06
col_18    2.355336e-06
col_14    2.355336e-06
col_16    2.355336e-06
col_11    2.355336e-06
col_12    2.355336e-06
col_8     2.899529e-06
col_3     3.018573e-06
col_7     8.099318e-05
col_10    8.283765e-05
col_19    1.142752e-04
col_1     1.636746e-04
col_5     2.148970e-01
col_6     2.325529e-01
col_4     2.856880e-01
dtype: float64

In [23]:
str(tmp_iter_std_dict)

"{'col_2': 4.5710077075063205e-08, 'col_13': 1.7940362261954181e-06, 'col_9': 2.28550385375316e-06, 'col_15': 2.2855038537531604e-06, 'col_17': 2.285503853753161e-06, 'col_18': 2.3553357890347472e-06, 'col_14': 2.355335789034749e-06, 'col_16': 2.3553357890347494e-06, 'col_11': 2.3553357890347494e-06, 'col_12': 2.3553357890347494e-06, 'col_8': 2.8995285030659137e-06, 'col_3': 3.018573450160837e-06, 'col_7': 8.099318038527539e-05, 'col_10': 8.283765119286316e-05, 'col_19': 0.00011427519268765797, 'col_1': 0.00016367458807929326, 'col_5': 0.21489700410384896, 'col_6': 0.23255291809784379, 'col_4': 0.28568798171914495}"

In [24]:
# 求交集
del_col = list(tmp_iter_std.iloc[:prepare_del_num].index & iter_left_series.index)

In [25]:
del_col

['col_2', 'col_13']

In [26]:
# del_col = list(tmp_iter_std.iloc[:2].index)

In [27]:
# del_col = del_col[:]

In [28]:
# 删除个数
del_len = len(del_col)

In [29]:
del_train_data = train_data.drop(del_col, axis=1)
del_test_data = test_data.drop(del_col, axis=1)
for del_k_value in range(1, 21):
    del_clf = knn.KNeighborsClassifier(n_neighbors = del_k_value, weights = 'uniform')
    del_clf.fit(del_train_data.drop('label', axis = 1), del_train_data['label'])
    # 预测结果
    del_predict_label_array = del_clf.predict(del_test_data.drop('label', axis=1))
    del_classic_knn_result = stat.get_accuracy_score(del_predict_label_array, del_test_data['label'])
    last_result_df.loc['{data}:{k}'.format(data = file_name, k = del_k_value), 'delKNN'] = round(del_classic_knn_result * 100,2)
    last_result_df.loc['{data}:{k}'.format(data = file_name, k = del_k_value), 'del_attr'] = del_col
    last_result_df.loc['{data}:{k}'.format(data = file_name, k = del_k_value), 'del_num'] = del_len
    last_result_df.loc['{data}:{k}'.format(data = file_name, k = del_k_value), 'pre_del_num'] = prepare_del_num
    last_result_df.loc['{data}:{k}'.format(data = file_name, k = del_k_value), 'iter_cdf_mean'] = str(cdf_cumsum_mean_dict)
    last_result_df.loc['{data}:{k}'.format(data = file_name, k = del_k_value), 'iter_std'] = str(tmp_iter_std_dict)

In [30]:
last_result_df

,k_value,KNN,delKNN,del_num,pre_del_num,del_attr,iter_cdf_mean,iter_std,data
new-segmentation:1,1,79.73,89.19,2,5,"[col_2, col_13]","{'col_4': 0.594576188030447, 'col_6': 0.832672...","{'col_2': 4.5710077075063205e-08, 'col_13': 1....",new-segmentation
new-segmentation:2,2,70.27,83.78,2,5,"[col_2, col_13]","{'col_4': 0.594576188030447, 'col_6': 0.832672...","{'col_2': 4.5710077075063205e-08, 'col_13': 1....",new-segmentation
new-segmentation:3,3,72.97,86.49,2,5,"[col_2, col_13]","{'col_4': 0.594576188030447, 'col_6': 0.832672...","{'col_2': 4.5710077075063205e-08, 'col_13': 1....",new-segmentation
new-segmentation:4,4,72.97,83.78,2,5,"[col_2, col_13]","{'col_4': 0.594576188030447, 'col_6': 0.832672...","{'col_2': 4.5710077075063205e-08, 'col_13': 1....",new-segmentation
new-segmentation:5,5,74.32,85.14,2,5,"[col_2, col_13]","{'col_4': 0.594576188030447, 'col_6': 0.832672...","{'col_2': 4.5710077075063205e-08, 'col_13': 1....",new-segmentation
new-segmentation:6,6,70.27,82.43,2,5,"[col_2, col_13]","{'col_4': 0.594576188030447, 'col_6': 0.832672...","{'col_2': 4.5710077075063205e-08, 'col_13': 1....",new-segmentation
new-segmentation:7,7,72.97,82.43,2,5,"[col_2, col_13]","{'col_4': 0.594576188030447, 'col_6': 0.832672...","{'col_2': 4.5710077075063205e-08, 'col_13': 1....",new-segmentation
new-segmentation:8,8,74.32,79.73,2,5,"[col_2, col_13]","{'col_4': 0.594576188030447, 'col_6': 0.832672...","{'col_2': 4.5710077075063205e-08, 'col_13': 1....",new-segmentation
new-segmentation:9,9,75.68,78.38,2,5,"[col_2, col_13]","{'col_4': 0.594576188030447, 'col_6': 0.832672...","{'col_2': 4.5710077075063205e-08, 'col_13': 1....",new-segmentation
new-segmentation:10,10,75.68,79.73,2,5,"[col_2, col_13]","{'col_4': 0.594576188030447, 'col_6': 0.832672...","{'col_2': 4.5710077075063205e-08, 'col_13': 1....",new-segmentation


In [31]:
# last_result_df.to_excel('{filename}_阈值_{arg}.xls'.format(filename = file_name, arg = tmp_args))